# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [52]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [53]:
# checking your current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # using os.path.join(root,'*') may include only hidden files having path .ipyn...../filename/
    file_path_list = glob.glob(os.path.join(filepath,'*'))
    print(*file_path_list, sep='\n')

/home/workspace
/home/workspace/event_data/2018-11-30-events.csv
/home/workspace/event_data/2018-11-23-events.csv
/home/workspace/event_data/2018-11-22-events.csv
/home/workspace/event_data/2018-11-29-events.csv
/home/workspace/event_data/2018-11-11-events.csv
/home/workspace/event_data/2018-11-14-events.csv
/home/workspace/event_data/2018-11-20-events.csv
/home/workspace/event_data/2018-11-15-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-28-events.csv
/home/workspace/event_data/2018-11-25-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-24-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-26-events.csv
/home/workspace/event_data/2018-11-12-events.csv
/home/workspace/event_data/2018-11-27-events.csv
/home/workspace/event_data/2018-11-06-events.csv
/hom

In [54]:
# Making sure all files are considered for loading
len(file_path_list)

30

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [55]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        #  extracting each data row one by one and append it    
        
        for line in csvreader:
            
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print("#"*50)
# print(len(full_data_rows_list))
# print("#"*50)

# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


##################################################
8056
##################################################


In [56]:
# check the number of rows in your csv file. The row count includes header row
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f)) 

6821


<font color = "blue"> The event_datafile_new.csv contains the following columns: </font>
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [57]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1']) # need locally installed instance of cassandra
    session = cluster.connect() # need a session
except Exception as e:
    print(e)

#### Create Keyspace

In [58]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS apachevikrant
    WITH REPLICATION = 
    {'class':'SimpleStrategy', 'replication_factor':1} """)
except Exception as e:
    print(e)

#### Set Keyspace

In [59]:
try:
    session.set_keyspace('apachevikrant')
except Exception as e:
    print(e)

<em>
<ins> **Given Query Need** </ins>

<font color = "brown"> **1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4** </font>


<font color = "brown"> **2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182** </font>
    

<font color = "brown"> **3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'** </font>

</em>


<br>

# CREATE AND TEST TABLE FOR QUERY 1

<hr>

<font color = "red">**For this query, sessionId AND itemInSession were used as a composite primary key because we needed to fetch the artist, song and song length based on a given session and item in session. This will result in the underlying data be stored on separate nodes separated by session id and item in session columns. This partitioning would help cassandra to know which node to go to based on the where condition in our CQL hence resulting in fast data retrieval.**</font> 

<hr>

In [60]:

query1 = """

CREATE TABLE IF NOT EXISTS artist_song_session (sessionId int, itemInSession int, artist text, song text, length float, \
                                            PRIMARY KEY (sessionId, itemInSession))

"""

try:
    session.execute(query1)
except Exception as e:
    print(e)

In [61]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))) # need to cast length as float due to csvreader output

#### Do a SELECT to verify that the data have been inserted into each table

In [63]:
query = "select artist, song, length, sessionid, iteminsession from artist_song_session where sessionId = 829 and itemInSession = 0"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length, row.sessionid, row.iteminsession)

Stephen Lynch Jim Henson's Dead 182.856689453125 829 0


<hr>

<br>

# CREATE AND TEST TABLE FOR QUERY 2

<font color = "red">**For this query, we will be using a composite key of userid and sessionid as our partition columns and include iteminsession as our clustering column. While the partition keys would split the dataset on different nodes making it easier for cassandra to identify a specific node based on our where condition, then clustering column will ensure results are returned in a sorted order(ascending) as needed.Unlike in query 1, here it makes more sense to use user id as one of our partition column as we need to fetch information based on a given user and session**</font> 

<hr>

</hr>

In [64]:

query2 = """

CREATE TABLE IF NOT EXISTS artist_song_user_session (userid int, sessionid int, itemInSession int, artist text, song text, first_name text, last_name text, \
                                            PRIMARY KEY ((userid, sessionid), itemInSession))

"""

try:
    session.execute(query2)
except Exception as e:
    print(e)

                    

In [65]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user_session (userid, sessionid, itemInSession, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])) # need to cast length as float due to csvreader output

In [66]:
query = "select artist, song, first_name, last_name from artist_song_user_session where userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


<hr>

<br>

# CREATE AND TEST TABLE FOR QUERY 3

<font color = "red">**For this query, we use song as the partition colum along with userid since we need to fetch user information based on the song they have/are listened/listening to. We add userid as a partition column to make each row/event uniquley identifiable.**</font> 

<hr>

In [68]:

query3 = """

CREATE TABLE IF NOT EXISTS user_by_song (song text, userid int, first_name text, last_name text,  \
                                           PRIMARY KEY (song, userid))

"""

try:
    session.execute(query3)
except Exception as e:
    print(e)

                    

In [69]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO user_by_song (song, userid, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4])) # need to cast length as float due to csvreader output

In [70]:
query = "select first_name, last_name, userid from user_by_song where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name, row.userid)

Jacqueline Lynch 29
Tegan Levine 80
Sara Johnson 95


### Drop the tables before closing out the sessions

In [74]:
##################################################
query_drop = "DROP TABLE artist_song_session"
try:
    rows = session.execute(query_drop)
except Exception as e:
    print(e)
    
##################################################

query_drop = "DROP TABLE artist_song_user_session"
try:
    rows = session.execute(query_drop)
except Exception as e:
    print(e)
    
##################################################
query_drop = "DROP TABLE user_by_song"
try:
    rows = session.execute(query_drop)
except Exception as e:
    print(e)
    

### Close the session and cluster connection¶

In [75]:
session.shutdown()
cluster.shutdown()